In [1]:
import pandas as pd
file_path = "/mnt/pfs-guan-ssai/nlu/zhangkehao/MemAgent_minimal/taskutils/memory_data/hotpotqa_dev.parquet"
df = pd.read_parquet(file_path)
# df['agent_name'] = 'mem_agent'
# df.to_parquet("/mnt/pfs-guan-ssai/nlu/zhangkehao/MemAgent_minimal/taskutils/memory_data/hotpotqa_dev_agent_loop.parquet")

In [1]:
import json
# datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/HippoRAG/longmemeval_s_r1_log_no_longcontext_recall20_eval_res.jsonl"
# datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/MemAgent_minimal/taskutils/memory_eval/results/longmemeval_s/Qwen3-8B-5k-1k-infty_eval_res.jsonl"
# datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/emergence_simple_fast/processing_log_qwen3-8b_1758104883.jsonl"
# datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/emergence_simple_fast/processing_log_qwen3-8b_faster_1758253073.jsonl"
# datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/results/qwen3-8b/longmemeval/evaluated_concat.jsonl"
datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/results/qwen3-8b/longmemeval_oracle/evaluated_concat.jsonl"
datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/ReMe/cookbook/simple_demo/longmemeval/evaluated_unknown.jsonl"
data = [json.loads(line) for line in open(datapath)]

original_datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/emergence_simple_fast/data/longmemeval_s.json"
original_data = json.load(open(original_datapath))
for d in data:
    for od in original_data:
        question_id = d.get("question_id", d.get("qid"))
        if "longmemeval_" in question_id:
            question_id = question_id[len("longmemeval_"):-2]
        if od["question_id"] == question_id:
            d["question_type"] = od["question_type"]
            break

# datapath = "longmemeval_s_r1_log_no_longcontext_recall20.json"
# data = json.load(open(datapath))
print("数据条数：" + str(len([x for x in data if x.get('qid') is not None])))
print("正确率：" + str(len([x for x in data if x["metric"].get('llm_score')]) / len([x for x in data if x["metric"].get('llm_score') is not None])))

for tp in ["knowledge-update", "single-session-user", "temporal-reasoning", "single-session-preference", "single-session-assistant", "multi-session"]:
    if len([x for x in data if x.get('question_type')==tp]) == 0:
        print(tp, 0)
    else:
        print(tp, len([x for x in data if x["metric"].get('llm_score') and x.get('question_type')==tp]) / len([x for x in data if x.get('question_type')==tp]))
        print(f"{len([x for x in data if x["metric"].get('llm_score') and x.get('question_type')==tp])} / {len([x for x in data if x.get('question_type')==tp])}")

数据条数：500
正确率：0.382
knowledge-update 0.4358974358974359
34 / 78
single-session-user 0.4142857142857143
29 / 70
temporal-reasoning 0.46616541353383456
62 / 133
single-session-preference 0.6
18 / 30
single-session-assistant 0.17857142857142858
10 / 56
multi-session 0.2857142857142857
38 / 133


In [4]:
import json

datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/results/qwen3-8b/locomo/evaluated_concat.jsonl"
datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/emergence_simple_fast/locomo_results_qwen3-8b_faster_1759028934.jsonl"
datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/emergence_simple_fast/locomo_results_1759036272.jsonl"
datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/MemAgent_minimal/taskutils/memory_eval/results/locomo/Qwen3-8B-5k-1k-infty.jsonl"
datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/HippoRAG/locomo_hippo_qwen_recall42.jsonl"
datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/HippoRAG/locomo_hippo_qwen_recall20.jsonl"
datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/ReMe/cookbook/simple_demo/locomo/evaluated_unknown.jsonl"
data = [json.loads(line) for line in open(datapath)]
# data = sum([d["queries_solutions"] for d in data], start=[])

original_datapath = "/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/raw/locomo10.json"
original_data = json.load(open(original_datapath))

for d in data:
    for od in original_data:
        if od['sample_id'] in d['qid']:
            q_index = int(d['qid'].split('_')[2])
            d['question_type'] = od['qa'][q_index]['category']

for d in data:
    # d['question_type'] = d['category']
    d["gpt4o_score"] =  d["metric"].get('llm_score')

print("数据条数：" + str(len(data)))
print("正确率：" + str(len([x for x in data if x["gpt4o_score"]]) / len([x for x in data if x["gpt4o_score"] is not None])))

question_type_map = {
    1: "multi hop",
    2: "temporal",
    3: "open domain",
    4: "single hop",
    5: "adversarial"
}


for tp in range(1,6):
    if len([x for x in data if x.get('question_type')==tp]) == 0:
        print(question_type_map[tp], 0)
    else:
        print(question_type_map[tp], len([x for x in data if x["gpt4o_score"] and x.get('question_type')==tp]) / len([x for x in data if x.get('question_type')==tp]))
        print(f"{len([x for x in data if x["gpt4o_score"] and x.get('question_type')==tp])} / {len([x for x in data if x.get('question_type')==tp])}")

数据条数：1986
正确率：0.28751258811681774
multi hop 0.09574468085106383
27 / 282
temporal 0.07476635514018691
24 / 321
open domain 0.21875
21 / 96
single hop 0.06539833531510107
55 / 841
adversarial 0.9955156950672646
444 / 446


In [ ]:
import json
import tqdm
from openai import OpenAI, RateLimitError
import uuid
import backoff
import asyncio
from tqdm.asyncio import tqdm as atqdm

from config import MODEL_NAME, API_CONFIG_LOCAL
from openai import OpenAI

async def gpt4o_evaluation(question, correct_answer, model_response, question_type):
    """
    Use GPT-4o to evaluate the model response
    Based on reprocess_res.py
    """

    API_CONFIG_LOCAL["base_url"] = "http://172.24.139.15:8000/v1"
    openai_client = OpenAI(**API_CONFIG_LOCAL)

    @backoff.on_exception(
        backoff.expo,
        RateLimitError,
        max_tries=16,
        max_time=300,
        jitter=backoff.full_jitter
    )
    async def llm_response(messages):
        # 使用 asyncio.to_thread 将同步调用转为异步
        res = await asyncio.to_thread(
            openai_client.chat.completions.create,
            model=MODEL_NAME,
            messages=messages,
            temperature=0,
            max_tokens=2048,
        )
        response_message = res.choices[0].message.content
        if "</think>" in response_message:
            response_message = response_message.split("</think>")[-1].strip()
        # print(response_message)
        return response_message
    
    def get_anscheck_prompt(task, question, answer, response):
        LLM_TEMPLATE = "I will give you a question, a correct answer, and a response from a model. Please answer yes if the response contains the correct answer. Otherwise, answer no. If the response is equivalent to the correct answer or contains all the intermediate steps to get the correct answer, you should also answer yes. If the response only contains a subset of the information required by the answer, answer no.\n\nQuestion: {}\n\nCorrect Answer: {}\n\nModel Response: {}\n\nIs the model response correct? Answer yes or no only."
        prompt = LLM_TEMPLATE.format(question, answer, response)
        return prompt
    
    async def anscheck(task, question, answer, response):
        prompt = get_anscheck_prompt(task, question, answer, response)
        messages = [
            {
                "role": "user",
                "content": prompt
            }
        ]
        res = await llm_response(messages)
        return 'yes' in res.lower() and not 'no' in res.lower()
    
    return await anscheck(question_type, question, correct_answer, model_response)

async def process_item(d, output_file, semaphore, file_lock):
    async with semaphore:
        score = await gpt4o_evaluation(d["input"], d["answer"], d["response"], d["question_type"])
        d["gpt4o_score"] = score
        
        # 使用文件锁确保并发写入安全
        async with file_lock:
            with open(output_file, "a") as f:
                f.write(json.dumps(d) + '\n')
        
        return d

async def main():
    data_path = "/mnt/pfs-guan-ssai/nlu/zhangkehao/MemAgent_minimal/taskutils/memory_eval/results/locomo/Qwen3-8B-5k-1k-infty.jsonl"
    data = [json.loads(line) for line in open(data_path)]
    
    output_file = "/mnt/pfs-guan-ssai/nlu/zhangkehao/MemAgent_minimal/taskutils/memory_eval/results/locomo/Qwen3-8B-5k-1k-infty_.jsonl"
    
    # 创建信号量控制并发数
    semaphore = asyncio.Semaphore(20)  # 可调整并发数
    
    # 创建文件锁，确保并发写入安全
    file_lock = asyncio.Lock()
    
    # 创建所有任务
    tasks = [process_item(d, output_file, semaphore, file_lock) for d in data]
    
    # 使用异步进度条执行
    results = await atqdm.gather(*tasks, desc="Processing")

if __name__ == "__main__":
    # asyncio.run(main())
    await main()

In [1]:
from config import MODEL_NAME, API_CONFIG_LOCAL, API_CONFIG
from openai import OpenAI
API_CONFIG_LOCAL["base_url"] = "http://172.24.139.15:8000/v1"
openai_client = OpenAI(**API_CONFIG_LOCAL)
# openai_client = OpenAI(**API_CONFIG)
messages = [
    {
        "role": "user",
        "content": "hello"
    }
]
res = openai_client.chat.completions.create(
    model="Qwen/Qwen3-4B-Instruct-2507",
    messages=messages,
    temperature=0,
    max_tokens=2048,
    logprobs=1
)
print(res.choices[0])

[EvalDataset] Registered benchmarks: banking77, booksum, clinic, convomem, hotpotqa, infbench, locomo, longmemeval, memalpha, msc, nlu, perltqa, pubmed_rct, squad, synth, trec_coarse, trec_fine
Choice(finish_reason='stop', index=0, logprobs=ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='Hello', bytes=[72, 101, 108, 108, 111], logprob=-0.04368528351187706, top_logprobs=[]), ChatCompletionTokenLogprob(token='!', bytes=[33], logprob=-1.0728830375228426e-06, top_logprobs=[]), ChatCompletionTokenLogprob(token=' How', bytes=[32, 72, 111, 119], logprob=-0.3034718334674835, top_logprobs=[]), ChatCompletionTokenLogprob(token=' can', bytes=[32, 99, 97, 110], logprob=-9.536697689327411e-06, top_logprobs=[]), ChatCompletionTokenLogprob(token=' I', bytes=[32, 73], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' assist', bytes=[32, 97, 115, 115, 105, 115, 116], logprob=-0.07889139652252197, top_logprobs=[]), ChatCompletionTokenLogprob(token=' you', bytes=[32, 121, 111, 1

In [3]:
res.choices[0]

Choice(finish_reason='stop', index=0, logprobs=ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='Hello', bytes=[72, 101, 108, 108, 111], logprob=-0.04368528351187706, top_logprobs=[]), ChatCompletionTokenLogprob(token='!', bytes=[33], logprob=-1.0728830375228426e-06, top_logprobs=[]), ChatCompletionTokenLogprob(token=' How', bytes=[32, 72, 111, 119], logprob=-0.3034718334674835, top_logprobs=[]), ChatCompletionTokenLogprob(token=' can', bytes=[32, 99, 97, 110], logprob=-9.536697689327411e-06, top_logprobs=[]), ChatCompletionTokenLogprob(token=' I', bytes=[32, 73], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' assist', bytes=[32, 97, 115, 115, 105, 115, 116], logprob=-0.07889139652252197, top_logprobs=[]), ChatCompletionTokenLogprob(token=' you', bytes=[32, 121, 111, 117], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=' today', bytes=[32, 116, 111, 100, 97, 121], logprob=-9.536738616588991e-07, top_logprobs=[]), ChatCompletionTokenLogprob(to

In [ ]:
[x.token for x in res.choices[0].logprobs.content]
res.choices[0].message.content

['<think>',
 '\n',
 'Okay',
 ',',
 ' the',
 ' user',
 ' said',
 ' "',
 'hello',
 '".',
 ' I',
 ' need',
 ' to',
 ' respond',
 ' appropriately',
 '.',
 ' Since',
 ' it',
 "'s",
 ' a',
 ' greeting',
 ',',
 ' I',
 ' should',
 ' acknowledge',
 ' it',
 ' and',
 ' offer',
 ' assistance',
 '.',
 ' Let',
 ' me',
 ' make',
 ' sure',
 ' the',
 ' response',
 ' is',
 ' friendly',
 ' and',
 ' open',
 '-ended',
 '.',
 ' Maybe',
 ' ask',
 ' how',
 ' I',
 ' can',
 ' help',
 ' them',
 ' today',
 '.',
 ' Keep',
 ' it',
 ' simple',
 ' and',
 ' welcoming',
 '.\n',
 '</think>',
 '\n\n',
 'Hello',
 '!',
 ' How',
 ' can',
 ' I',
 ' assist',
 ' you',
 ' today',
 '?',
 ' �',
 '�',
 '<|im_end|>']

In [1]:
import json
def load_context() -> str:
    """Load contextual information fed into ``ToolMemoryAgentLoop``.

    Projects can override this helper to supply domain-specific context. By default it
    returns the value from ``TOOL_MEM_AGENT_TEST_CONTEXT`` (or an empty string).
    """

    # return os.environ.get("TOOL_MEM_AGENT_TEST_CONTEXT", "I like to eat bananas.")
    raw = json.load(open("/mnt/pfs-guan-ssai/nlu/zhangkehao/zep/benchmarks/longmemeval/data/longmemeval_s.json"))
    item0 = raw[0]
    context = "Below is a conversation between User and Assistant.\n\n"
    for idx, session in enumerate(item0["haystack_sessions"]):
        session_date = item0["haystack_dates"][idx]
        session_conv = ''
        for turn in session:
            role = "User" if turn['role'] == 'user' else "Assistant"
            turn_text = f'{role} said, "{turn["content"]}"'
            turn_text += '\n'
            session_conv += turn_text
        if not session_conv:
            session_conv = "NO CONVERSATION"
        query_conv = f'DATE: {session_date}\nCONVERSATION:\n{session_conv}\n\n'
        context += query_conv
    return context

In [ ]:
# reformat
import json
input_file = "/mnt/pfs-guan-ssai/nlu/zhangkehao/verl/base_result.jsonl"
output_file = "/mnt/pfs-guan-ssai/nlu/zhangkehao/verl/base_result_reformat.jsonl"
original_file = "/mnt/pfs-guan-ssai/nlu/zhangkehao/zep/benchmarks/longmemeval/data/longmemeval_s.json"
data = [json.loads(line) for line in open(input_file)]
original_data = json.load(open(original_file))
with open(output_file, "w") as f:
    for d in data:
        original_d = original_data[d['idx']]
        response = d["response"]
        if "\\boxed{" in response and "}" in response.split("\\boxed{")[-1]:
            response = response.split("\\boxed{")[-1]
            response = response[:response.rindex("}")]
        new_d = {
            "qid": "longmemeval_" + original_d["question_id"] + "_0",
            "query": original_d["question"],
            "expected_answer": original_d["answer"],
            "response": response
        }
        f.write(json.dumps(new_d) + '\n')

In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
memalpha_file = '/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/memalpha_dev_verl.parquet'
locomo_file = '/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/locomo_train_verl.parquet'
hpq_file = '/mnt/pfs-guan-ssai/nlu/zhangkehao/MemAgent_minimal/taskutils/memory_data/hotpotqa_train_mem_agent_loop_chunk2000.parquet'
sync_file = '/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/synth_train_verl.parquet'
for input_file, name in zip([memalpha_file, locomo_file, hpq_file, sync_file], ['memalpha', 'locomo', 'hotpotqa', 'synth']):
    print(name)
    df = pd.read_parquet(input_file)
    print(len(df), 'records')
    print(np.mean([len([len(x) for x in y['tools_kwargs']['memory_bm25_retrieve']['create_kwargs']['chunks']]) for y in df['extra_info']]), 'chunks') # 平均chunks数量
    print(np.mean([len(x) for y in df['extra_info'] for x in y['tools_kwargs']['memory_bm25_retrieve']['create_kwargs']['chunks']]), 'chars\n') # 平均每个chunk字符数量

memalpha
1461 records
9.89596167008898 chunks
4046.8266011896526 chars

locomo
20 records
19.0 chunks
3942.8947368421054 chars

hotpotqa
8192 records
61.06103515625 chunks
1701.0528355977065 chars

synth
89 records
10.0 chunks
6008.816853932584 chars



In [1]:
import os
os.environ["HF_HOME"] = "/mnt/pfs-guan-ssai/nlu/zhangkehao/.cache/huggingface"
import json
import numpy as np
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-4B-Instruct-2507", use_fast=True)
benchmark_files = {
    'hotpotqa_file' : '/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/processed_hotpotqa_200.json',
    'locomo_file' : '/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/processed_locomo.json',
    'longmemeval_file' : '/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/processed_longmemeval.json',
    'msc_file' : '/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/processed_msc_batch5.json',
    'banking77_file' : '/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/processed_banking77.json',
    'booksum_file' : '/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/processed_booksum.json',
    'clinic_file' : '/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/processed_clinic.json',
    'memalpha_file' : '/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/processed_memalpha.json',
    'nlu_file' : '/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/processed_nlu.json',
    'perltqa_file' : '/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/processed_perltqa.json',
    'pubmed_rct_file' : '/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/processed_pubmed_rct.json',
    'trec_coarse_file' : '/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/processed_trec_coarse.json',
    'trec_fine_file' : '/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/processed_trec_fine.json',
    'synth10_file' : '/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/processed_synth-s10.json',
    'synth50_file' : '/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/processed_synth-s50.json',
    'convomem_file' : '/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/processed_convomem.json',
}
for name, input_file in benchmark_files.items():
    print(name)
    data = json.load(open(input_file))
    print("数据条数：" + str(len(data)))
    print("平均问题数量：" + str(sum([len(item['questions']) for item in data]) / len(data)))
    print("平均chunks数量: " + str(np.mean([len(item['chunks']) for item in data])) + ' chunks')
    print("平均每个chunk字符数量: " + str(np.mean([len(chunk) for item in data for chunk in item['chunks']])) + ' chars')
    print("平均每个chunk token数量: " + str(np.mean([len(tokenizer.tokenize(chunk)) for item in data for chunk in item['chunks']])) + ' tokens')
    print("平均context总字符长度: " + str(np.mean([len(''.join(item['chunks'])) for item in data])) + ' chars')
    print()

hotpotqa_file
数据条数：128
平均问题数量：1.0
平均chunks数量: 200.0 chunks
平均每个chunk字符数量: 570.0202734375 chars
平均每个chunk token数量: 140.5790625 tokens
平均context总字符长度: 114004.0546875 chars

locomo_file
数据条数：10
平均问题数量：198.6
平均chunks数量: 27.2 chunks
平均每个chunk字符数量: 3398.1507352941176 chars
平均每个chunk token数量: 814.7279411764706 tokens
平均context总字符长度: 92429.7 chars

longmemeval_file
数据条数：500
平均问题数量：1.0
平均chunks数量: 47.764 chunks
平均每个chunk字符数量: 10521.834394104346 chars
平均每个chunk token数量: 2258.246838623231 tokens
平均context总字符长度: 502564.898 chars

msc_file
数据条数：100
平均问题数量：5.0
平均chunks数量: 25.0 chunks
平均每个chunk字符数量: 1563.9144 chars
平均每个chunk token数量: 394.1004 tokens
平均context总字符长度: 39097.86 chars

banking77_file
数据条数：1
平均问题数量：100.0
平均chunks数量: 111.0 chunks
平均每个chunk字符数量: 4279.54954954955 chars
平均每个chunk token数量: 1150.2522522522522 tokens
平均context总字符长度: 475030.0 chars

booksum_file
数据条数：155
平均问题数量：1.0
平均chunks数量: 8.135483870967741 chunks
平均每个chunk字符数量: 7643.934179222839 chars
平均每个chunk token数量: 1914.3243457573355 tok

In [80]:
def calculate_value(row):
    row['extra_info']['num_chunks'] = row['extra_info'].pop('num_docs')
    row['extra_info']['question'] = [row['extra_info']['question']]
    return row
df = df.apply(calculate_value, axis=1)
df.to_parquet(output_file)

In [5]:
df['agent_name'] = 'tool_mem_agent'
def get_chunks(context_text, chunk_size):
    separator = "\n"
    small_chunks = context_text.split(separator)
    chunks = []
    to_append = ""
    for chunk in small_chunks:
        if len(to_append) + len(chunk) + len(separator) <= chunk_size:
            if to_append:
                to_append += separator + chunk
            else:
                to_append = chunk
        else:
            if to_append:
                chunks.append(to_append)
            to_append = chunk
    if to_append:
        chunks.append(to_append)
    return chunks

def calculate_value(row):
    idx = row['extra_info']['index']
    filename = f"/mnt/pfs-guan-ssai/nlu/zhangkehao/verl/memagent/store/memory_store_val{idx}.jsonl"
    chunks = get_chunks(row['context'], 2000)
    row['extra_info']["tools_kwargs"] = {
        "memory_add": {
            "create_kwargs": {"filename": filename}
        },
        "memory_update": {
            "create_kwargs": {"filename": filename}
        },
        "memory_delete": {
            "create_kwargs": {"filename": filename}
        },
        "memory_key_retrieve": {
            "create_kwargs": {"filename": filename}
        },
        "memory_list": {
            "create_kwargs": {"filename": filename}
        },
        "memory_bm25_retrieve": {
            "create_kwargs": {"chunks": chunks}
        },
        "memory_embedding_retrieve": {
            "create_kwargs": {"chunks": chunks}
        },
    }
    return row['extra_info']

df['extra_info'] = df.apply(calculate_value, axis=1)
df.to_parquet(output_file)

In [2]:
import pandas as pd
MEMALPHA_PARQUET_PATH = "/mnt/pfs-guan-ssai/nlu/zhangkehao/Mem-alpha/data/memalpha/test.parquet"
MEMORYAGENTBENCH_PARQUET_PATH = "/mnt/pfs-guan-ssai/nlu/zhangkehao/Mem-alpha/data/memoryagentbench/test.parquet"
memalpha_df = pd.read_parquet(MEMALPHA_PARQUET_PATH)
memoryagentbench_df = pd.read_parquet(MEMORYAGENTBENCH_PARQUET_PATH)
print("Mem-alpha data_sorce sets:", memalpha_df['data_source'].unique())
print()
print("MemoryAgentBench data_sorce sets:", memoryagentbench_df['data_source'].unique())

Mem-alpha data_sorce sets: ['pubmed-rct' 'squad' 'hotpotqa' 'perltqa'
 'icl_trec_coarse_6600shot_balance' 'icl_nlu_8296shot_balance' 'booksum']

MemoryAgentBench data_sorce sets: ['ruler_qa1_197K' 'ruler_qa2_421K' 'longmemeval_s*'
 'infbench_sum_eng_shots2' 'icl_banking77_5900shot_balance'
 'icl_clinic150_7050shot_balance' 'icl_nlu_8296shot_balance'
 'icl_trec_coarse_6600shot_balance' 'icl_trec_fine_6400shot_balance']


In [3]:
import pandas as pd
set(pd.read_parquet("/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/synth_train_verl.parquet")['data_source']) | set(pd.read_parquet("/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/locomo_train_verl.parquet")['data_source']) | set(pd.read_parquet("/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/memalpha_train_verl.parquet")['data_source'])

{'locomo',
 'memalpha_booksum',
 'memalpha_hotpotqa',
 'memalpha_icl_nlu_8296shot_balance',
 'memalpha_icl_trec_coarse_6600shot_balance',
 'memalpha_perltqa',
 'memalpha_pubmed-rct',
 'memalpha_squad',
 'synth'}

In [7]:
import pandas as pd
set(pd.read_parquet("/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/memalpha_dev_verl.parquet")['data_source'])

{'memalpha_booksum',
 'memalpha_hotpotqa',
 'memalpha_icl_nlu_8296shot_balance',
 'memalpha_icl_trec_coarse_6600shot_balance',
 'memalpha_perltqa',
 'memalpha_pubmed-rct',
 'memalpha_squad'}

In [8]:
import pandas as pd
input_path = "/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/memalpha_dev_verl.parquet"
output_path = "/mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/memalpha_dev_verl_small.parquet"
df = pd.read_parquet(input_path)
sampled = (
    df.groupby("data_source", group_keys=False)
    .apply(lambda group: group.sample(n=20, random_state=42) if len(group) >= 20 else group)
    .reset_index(drop=True)
 )  # sample per source or keep all if fewer than 20
sampled.to_parquet(output_path)
print(f"Saved {len(sampled)} rows from {sampled['data_source'].nunique()} sources to {output_path}")

Saved 140 rows from 7 sources to /mnt/pfs-guan-ssai/nlu/zhangkehao/unified-memory-agent/data/memalpha_dev_verl_small.parquet


/tmp/ipykernel_993597/476900753.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.sample(n=20, random_state=42) if len(group) >= 20 else group)


In [12]:
from pathlib import Path
import json
base_dir = Path("/mnt/pfs-guan-ssai/nlu/zhangkehao/ConvoMem/ConvoMem/core_benchmark/pre_mixed_testcases")
json_files = sorted(p.as_posix() for p in base_dir.rglob("*.json"))

print(f"Found {len(json_files)} JSON files")

lengths = []
for path in json_files:
    total_data = json.load(open(path))
    for item in total_data:
        context = '\n'.join([mess['text'] for conv in item['conversations'] for mess in conv['messages']])
        lengths.append(len(context))

print(max(lengths))
print(len(lengths))
print(len([x for x in lengths if x >  1000000]))

Found 825 JSON files
2359440
243830
1339


In [11]:
len(total_data[0]['evidenceItems'])

50

In [5]:
from pathlib import Path
import json
base_dir = Path("/mnt/pfs-guan-ssai/nlu/zhangkehao/ConvoMem/ConvoMem/core_benchmark/evidence_questions")
json_files = sorted(p.as_posix() for p in base_dir.rglob("*.json"))

print(f"Found {len(json_files)} JSON files")

lengths = []
for path in json_files:
    total_data = json.load(open(path))
    for item in total_data['evidence_items']:
        context = '\n'.join([mess['text'] for conv in item['conversations'] for mess in conv['messages']])
        lengths.append(len(context))

print(max(lengths))
print(len(lengths))

Found 1242 JSON files
40590
75336


In [16]:
print(context)

Hey, I'm trying to finalize my fitness schedule for the week. Can you help me out?
Of course! What activities are you planning to include?
Well, climbing is definitely on the agenda. It's such a great workout and stress reliever.
Climbing sounds exciting! Do you have a specific day in mind for it?
Okay, let's lock in my climbing for the week. I'm going to hit Crux Climbing Center on Tuesday evening.
Tuesday evening at Crux Climbing Center it is. Anything else you'd like to add to your schedule?
Not at the moment, but I'll let you know if anything else comes up. Thanks for the help!
You're welcome! Let me know if you need any more assistance with your schedule.
I've been feeling a bit worn out lately. I think I need to plan some downtime.
It's important to listen to your body. Do you want to schedule a rest day?
Yes, definitely. I need to make sure I schedule a full rest day. My body is telling me it needs one. Let's block out this Wednesday for recovery.
Wednesday is now your designate

In [2]:
import pandas as pd
df = pd.read_parquet("/mnt/pfs-guan-ssai/nlu/zhangkehao/MemAgent_minimal/taskutils/memory_data/hotpotqa_train_mem_agent_loop_chunk2000.parquet")
len(df)
# df = pd.read_parquet("/mnt/pfs-guan-ssai/nlu/zhangkehao/MemAgent_minimal/taskutils/memory_data/hotpotqa_dev_agent_loop.parquet")
# def reformat(row):
#     row['extra_info']['question'] = [row['extra_info']['question']]
#     return row
# df = df.apply(reformat, axis=1)
# df.to_parquet("/mnt/pfs-guan-ssai/nlu/zhangkehao/MemAgent_minimal/taskutils/memory_data/hotpotqa_dev_agent_loop.parquet")

8192

In [ ]:
df.iloc[0]['extra_info']['tools_kwargs']

{'memory_add': {'create_kwargs': {'filename': '/mnt/pfs-guan-ssai/nlu/zhangkehao/verl/memagent/store/memory_store.jsonl'}},
 'memory_bm25_retrieve': {'create_kwargs': {'chunks': array(["[Dialogue between User and Assistant on 2024-01-01 00:00]\n<User>Here are some annotated classification examples:\nSentence: what movie should i see\nLabel: 37\n\nSentence: will it rain on Saturday\nLabel: 9\n\nSentence: what's my stock fluctuations\nLabel: 4\n\nSentence: send the email to this new email address\nLabel: 50\n\nSentence: that's all, stop it.\nLabel: 56\n\nSentence: add jpearsonjessica@gmail dot com to email contacts\nLabel: 1\n\nSentence: whats my schedule today\nLabel: 67\n\nSentence: rise volume\nLabel: 31\n\nSentence: how is shell s stock doing today\nLabel: 4\n\nSentence: could you please further explain.\nLabel: 16\n\nSentence: i want to listen to heavy metal please\nLabel: 17\n\nSentence: will you check the detail please.\nLabel: 12\n\nSentence: switch on the plug\nLabel: 34\n\nSent

In [11]:
from collections import Counter
counter = Counter()
for source in df['data_source']:
    counter[source] += 1
counter

Counter({'hotpotqa': 100,
         'squad': 100,
         'booksum': 100,
         'pubmed-rct': 90,
         'icl_trec_coarse_6600shot_balance': 51,
         'lme_train': 50,
         'icl_nlu_8296shot_balance': 49,
         'perltqa': 27})

In [14]:
sum(counter.values())

567

In [6]:
df

,data_source,prompt,context,reward_model,extra_info,agent_name
0,memalpha_icl_nlu_8296shot_balance,[{'content': ['Sentence: fight me in street fi...,I will provide you with classification trainin...,"{'ground_truth': ['43', '66', '5', '14', '15',...","{'index': 0, 'num_chunks': 10, 'original_sampl...",tool_mem_agent
1,memalpha_icl_nlu_8296shot_balance,[{'content': ['Sentence: what do you want to d...,I will provide you with classification trainin...,"{'ground_truth': ['66', '38', '5', '5', '5', '...","{'index': 1, 'num_chunks': 10, 'original_sampl...",tool_mem_agent
2,memalpha_icl_nlu_8296shot_balance,[{'content': ['Sentence: what are the bbc poll...,I will provide you with classification trainin...,"{'ground_truth': ['36', '38', '14', '18', '65'...","{'index': 2, 'num_chunks': 10, 'original_sampl...",tool_mem_agent
3,memalpha_icl_nlu_8296shot_balance,[{'content': ['Sentence: add eggs to my grocer...,I will provide you with classification trainin...,"{'ground_truth': ['57', '66', '21', '12', '29'...","{'index': 3, 'num_chunks': 10, 'original_sampl...",tool_mem_agent
4,memalpha_icl_nlu_8296shot_balance,[{'content': ['Sentence: send an email to emma...,I will provide you with classification trainin...,"{'ground_truth': ['50', '36', '23', '17', '41'...","{'index': 4, 'num_chunks': 10, 'original_sampl...",tool_mem_agent
...,...,...,...,...,...,...
3429,memalpha_pubmed-rct,[{'content': ['Sentence: It appears that poten...,You are a scientific document classifier. I wi...,"{'ground_truth': ['1', '4', '2', '3', '4', '0'...","{'index': 3429, 'num_chunks': 10, 'original_sa...",tool_mem_agent
3430,memalpha_pubmed-rct,[{'content': ['Sentence: Oxygen extraction rat...,You are a scientific document classifier. I wi...,"{'ground_truth': ['4', '0', '2', '4', '2', '4'...","{'index': 3430, 'num_chunks': 10, 'original_sa...",tool_mem_agent
3431,memalpha_pubmed-rct,[{'content': ['Sentence: The differences betwe...,You are a scientific document classifier. I wi...,"{'ground_truth': ['4', '3', '4', '2', '0', '2'...","{'index': 3431, 'num_chunks': 10, 'original_sa...",tool_mem_agent
3432,memalpha_pubmed-rct,[{'content': ['Sentence: We studied 137 patien...,You are a scientific document classifier. I wi...,"{'ground_truth': ['2', '1', '0', '2', '1', '4'...","{'index': 3432, 'num_chunks': 10, 'original_sa...",tool_mem_agent


In [7]:
from collections import Counter
counter = Counter()
for source in df['data_source']:
    counter[source] += 1
counter

Counter({'memalpha_squad': 957,
         'memalpha_pubmed-rct': 900,
         'memalpha_icl_trec_coarse_6600shot_balance': 510,
         'memalpha_icl_nlu_8296shot_balance': 490,
         'memalpha_perltqa': 270,
         'memalpha_hotpotqa': 207,
         'memalpha_booksum': 100})